#라이브러리 및 seed고정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [ ]:
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)
print("Scikit-Learn version:", sklearn.__version__)

Pandas version: 2.0.3
NumPy version: 1.25.2
Scikit-Learn version: 1.2.2


In [ ]:
view_log= pd.read_csv('./data/view_log.csv')
article_info = pd.read_csv('./data/article_info.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

# 모델링

In [ ]:
# 사용자-기사 행렬 생성
user_article_matrix = view_log.groupby(['userID', 'articleID']).size().unstack(fill_value=0)

# 사용자 간의 유사성 계산 (협업 필터링)
user_similarity = cosine_similarity(user_article_matrix)

# 협업 필터링을 기반으로 한 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_article_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T


In [ ]:
# 영어와 포르투갈어의 불용어 리스트
english_stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
portuguese_stop_words = ["a", "à", "ao", "aos", "aquela", "aquelas", "aquele", "aqueles", "aquilo", "as", "até", "com", "como", "da", "das", "de", "dela", "delas", "dele", "deles", "depois", "do", "dos", "e", "ela", "elas", "ele", "eles", "em", "entre", "era", "eram", "essa", "essas", "esse", "esses", "esta", "está", "estão", "estas", "estava", "estavam", "este", "estes", "eu", "foi", "foram", "fui", "há", "isso", "isto", "já", "lhe", "lhes", "mas", "me", "mesmo", "meu", "meus", "minha", "minhas", "muito", "na", "não", "nas", "nem", "no", "nos", "nossa", "nossas", "nosso", "nossos", "num", "numa", "o", "os", "ou", "para", "pela", "pelas", "pelo", "pelos", "por", "qual", "quando", "que", "quem", "se", "seu", "seus", "só", "suas", "também", "te", "tem", "tinha", "tive", "tivemos", "tiveram", "tua", "tuas", "tudo", "um", "uma", "você", "vocês"]

# 영어와 포르투갈어 불용어를 합친 사용자 정의 불용어 리스트 생성
custom_stop_words = list(english_stop_words) + list(portuguese_stop_words)

# 콘텐츠 기반 필터링을 위한 TF-IDF 행렬 생성
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(article_info['Content'])
cosine_sim_content = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim_content, index=article_info['articleID'], columns=article_info['articleID'])


In [ ]:
# 사용자-기사 행렬에서 사용자의 조회 기록을 활용해 콘텐츠 기반 추천 점수 계산
def calculate_content_based_scores(user_article_matrix, cosine_sim_df):
    content_based_scores = np.zeros(user_article_matrix.shape)
    for idx, user in enumerate(user_article_matrix.index):
        user_history = user_article_matrix.loc[user]
        user_articles = user_history[user_history > 0].index
        if len(user_articles) > 0:
            user_sim_scores = cosine_sim_df.loc[user_articles].mean(axis=0)
            content_based_scores[idx] = user_sim_scores.reindex(user_article_matrix.columns, fill_value=0).values
    return content_based_scores

content_based_scores = calculate_content_based_scores(user_article_matrix, cosine_sim_df)


In [ ]:
# 협업 필터링과 콘텐츠 기반 필터링 점수 결합
combined_scores = user_predicted_scores + content_based_scores

In [ ]:
# 자신의 기사를 추천 목록에 포함시키기 위해 view_log와 article_info 병합
merged_df = pd.merge(view_log, article_info, on='articleID', suffixes=('_view', '_article'))

In [ ]:
# 자신의 기사를 읽은 유저 필터링
self_viewed_articles = merged_df[merged_df['userID_view'] == merged_df['userID_article']]

# 각 사용자가 읽은 기사 수 계산
user_article_count = self_viewed_articles.groupby('userID_view').size()

In [ ]:
# 추천 목록을 생성하는 함수
def generate_recommendations(user_article_matrix, combined_scores, merged_df, user_article_count):
    recommendations = []
    for idx, user in enumerate(user_article_matrix.index):
        # 사용자가 작성한 기사 추가
        user_written_articles = merged_df[merged_df['userID_article'] == user][['articleID', 'userID_view']]

        # 각 기사별 조회 수 계산
        article_view_counts = user_written_articles['articleID'].value_counts().reset_index()
        article_view_counts.columns = ['articleID', 'view_count']
        article_view_counts = article_view_counts.sort_values(by='view_count', ascending=False)

        # 포함할 기사 수 결정
        if user in user_article_count.index:
            num_self_articles = user_article_count[user]
            if num_self_articles >= user_article_count.quantile(0.9):
                max_self_articles = 2
            else:
                max_self_articles = 1
        else:
            max_self_articles = 1

        top_user_written_articles = article_view_counts['articleID'][:max_self_articles].tolist()

        # 사용자의 상위 추천 가져오기
        sorted_indices = combined_scores[idx].argsort()[::-1]
        top_recommendations = [article for article in user_article_matrix.columns[sorted_indices]]

        # 사용자가 여러 번 읽은 기사 추가
        user_view_counts = view_log[view_log['userID'] == user]['articleID'].value_counts()
        repeated_articles = user_view_counts[user_view_counts > 1].index.tolist()

        # 추천 목록 결합
        unique_recommendations = list(dict.fromkeys(repeated_articles + top_user_written_articles + [article for article in top_recommendations if article not in repeated_articles and article not in top_user_written_articles]))

        # 상위 5개 추천 선택
        top5recommend = unique_recommendations[:5]

        for article in top5recommend:
            recommendations.append([user, article])
    return recommendations

# 추천 생성
recommendations = generate_recommendations(user_article_matrix, combined_scores, merged_df, user_article_count)


In [ ]:
# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

# 제출 파일 생성
submission = sample_submission.copy()
submission['articleID'] = top_recommendations['articleID']

submission.to_csv('./data/final_submission.csv', index=False)
submission

,userID,articleID
0,USER_0000,ARTICLE_2255
1,USER_0000,ARTICLE_0411
2,USER_0000,ARTICLE_1345
3,USER_0000,ARTICLE_1033
4,USER_0000,ARTICLE_1260
...,...,...
7070,USER_1420,ARTICLE_0030
7071,USER_1420,ARTICLE_0714
7072,USER_1420,ARTICLE_0614
7073,USER_1420,ARTICLE_0089


In [ ]:
# 제출 파일 다운로드
#from google.colab import files
#files.download('./data/final_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 추천 목록 내 중복확인

In [ ]:
# 각 사용자의 추천 목록 내 중복 확인
duplicates_within_user = submission.groupby('userID')['articleID'].apply(lambda x: x.duplicated()).sum()
duplicates_within_user

# 몇몇 사용자의 추천 검사
inspection = submission.groupby('userID').apply(lambda x: x['articleID'].duplicated().sum()).reset_index(name='duplicate_count')
inspection.head()

# 중복 추천이 있는 사용자가 있는 경우 표시
users_with_duplicates = inspection[inspection['duplicate_count'] > 0]
users_with_duplicates


,userID,duplicate_count
